In [18]:
import numpy as np
import pandas as pd

In [19]:
df = pd.read_csv("preprocessed_clv_dataset.csv")

In [20]:
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [21]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [22]:
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


# RFM Analysis


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359450 entries, 0 to 359449
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      359450 non-null  int64  
 1   StockCode    359450 non-null  object 
 2   Quantity     359450 non-null  int64  
 3   InvoiceDate  359450 non-null  object 
 4   Price        359450 non-null  float64
 5   Customer ID  359450 non-null  float64
 6   Country      359450 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 19.2+ MB


In [23]:
cutoff_date = pd.Timestamp("2010-07-01")

In [24]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359450 entries, 0 to 359449
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      359450 non-null  int64         
 1   StockCode    359450 non-null  object        
 2   Quantity     359450 non-null  int64         
 3   InvoiceDate  359450 non-null  datetime64[ns]
 4   Price        359450 non-null  float64       
 5   Customer ID  359450 non-null  float64       
 6   Country      359450 non-null  object        
 7   TotalPrice   359450 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 21.9+ MB


In [25]:
obs_df = df[df["InvoiceDate"] <= cutoff_date]
pred_df = df[df["InvoiceDate"] > cutoff_date]

In [26]:
customer_grp = obs_df.groupby('Customer ID')

In [27]:
customer_grp.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0
...,...,...,...,...,...,...,...,...
195217,514208,21687,4,2010-06-30 16:35:00,3.75,13649.0,United Kingdom,15.0
195218,514208,21692,4,2010-06-30 16:35:00,3.75,13649.0,United Kingdom,15.0
195219,514208,21696,6,2010-06-30 16:35:00,2.95,13649.0,United Kingdom,17.7
195220,514208,21671,12,2010-06-30 16:35:00,1.25,13649.0,United Kingdom,15.0


# Recency
## How long since the customer purchased
Low Recency == better customer

In [28]:
Recency = (cutoff_date - customer_grp['InvoiceDate'].max()).dt.days
Recency

,InvoiceDate
Customer ID,
12346.0,2
12349.0,43
12355.0,40
12358.0,23
12359.0,8
...,...
18281.0,50
18283.0,94
18285.0,133


# Frequency
It's show engagement  lable how many purchase customer does with the company

In [29]:
frequency = customer_grp['InvoiceDate'].nunique()
frequency

,InvoiceDate
Customer ID,
12346.0,11
12349.0,2
12355.0,1
12358.0,2
12359.0,5
...,...
18281.0,1
18283.0,3
18285.0,1


# Monetary Value
## How much money the customer spend

It directly influence the CLV

In [30]:
monetary = customer_grp['TotalPrice'].sum()
monetary

,TotalPrice
Customer ID,
12346.0,372.86
12349.0,1268.52
12355.0,488.21
12358.0,1697.93
12359.0,2012.03
...,...
18281.0,120.32
18283.0,354.42
18285.0,427.00


# Average Monetary Vlaue(AOV)

Spending Quality Per order

In [31]:
aov = monetary / frequency
aov

,0
Customer ID,
12346.0,33.896364
12349.0,634.260000
12355.0,488.210000
12358.0,848.965000
12359.0,402.406000
...,...
18281.0,120.320000
18283.0,118.140000
18285.0,427.000000


# Tenure
How long they have been with the company


In [32]:
tenure = (cutoff_date - customer_grp['InvoiceDate'].min()).dt.days
tenure

,InvoiceDate
Customer ID,
12346.0,198
12349.0,62
12355.0,40
12358.0,204
12359.0,207
...,...
18281.0,50
18283.0,131
18285.0,133


# Purchase span
Activity spread

In [33]:
purchase_span = (customer_grp['InvoiceDate'].max()-customer_grp['InvoiceDate'].min()).dt.days
purchase_span

,InvoiceDate
Customer ID,
12346.0,196
12349.0,18
12355.0,0
12358.0,181
12359.0,198
...,...
18281.0,0
18283.0,36
18285.0,0


In [34]:
orders_per_month = frequency /(tenure / 30 + 1)
orders_per_month

,InvoiceDate
Customer ID,
12346.0,1.447368
12349.0,0.652174
12355.0,0.428571
12358.0,0.256410
12359.0,0.632911
...,...
18281.0,0.375000
18283.0,0.559006
18285.0,0.184049


In [35]:
revenue_per_month = monetary /(tenure / 30 + 1)
revenue_per_month

,0
Customer ID,
12346.0,49.060526
12349.0,413.647826
12355.0,209.232857
12358.0,217.683333
12359.0,254.687342
...,...
18281.0,45.120000
18283.0,66.040994
18285.0,78.588957


# Now all these are one customers specific data of 8 months of dataset
By these make a customer features dataset


In [37]:
customer_features = pd.DataFrame({
    "recency_days": Recency,
    "frequency": frequency,
    "monetary": monetary,
    "aov": aov,
    "tenure_days": tenure,
    "purchase_span_days": purchase_span,
    "orders_per_month": orders_per_month,
    "revenue_per_month": revenue_per_month
}).reset_index()


In [38]:
customer_features.isnull().sum()
customer_features['Customer ID'].duplicated().sum()

np.int64(0)

# Now saving the customer features into a new dataset

In [39]:
customer_features.to_csv(
    "customer_features.csv",
    index=False
)


In [40]:
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2975 entries, 0 to 2974
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Customer ID         2975 non-null   float64
 1   recency_days        2975 non-null   int64  
 2   frequency           2975 non-null   int64  
 3   monetary            2975 non-null   float64
 4   aov                 2975 non-null   float64
 5   tenure_days         2975 non-null   int64  
 6   purchase_span_days  2975 non-null   int64  
 7   orders_per_month    2975 non-null   float64
 8   revenue_per_month   2975 non-null   float64
dtypes: float64(5), int64(4)
memory usage: 209.3 KB


# CLV Label

In [41]:
pred_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
195283,514217,20658,12,2010-07-01 08:39:00,1.25,17029.0,United Kingdom,15.00
195284,514217,21155,6,2010-07-01 08:39:00,2.10,17029.0,United Kingdom,12.60
195285,514217,21035,6,2010-07-01 08:39:00,2.95,17029.0,United Kingdom,17.70
195286,514217,21498,25,2010-07-01 08:39:00,0.42,17029.0,United Kingdom,10.50
195287,514217,21936,5,2010-07-01 08:39:00,2.95,17029.0,United Kingdom,14.75


In [42]:
clv_df = pred_df.groupby('Customer ID')['TotalPrice'].sum().reset_index()
clv_df.head()

,Customer ID,TotalPrice
0,12347.0,611.53
1,12348.0,222.16
2,12349.0,1402.62
3,12352.0,143.75
4,12353.0,317.76


In [43]:
clv_df.rename(columns={"TotalPrice":"clv_4m"},inplace=True)

In [44]:
clv_df.to_csv('clv_labels.csv', index=False)

In [45]:
features = pd.read_csv('customer_features.csv')
label = pd.read_csv('clv_labels.csv')

In [46]:
model_df = features.merge(label, on='Customer ID', how="left")

In [47]:
model_df.head()

,Customer ID,recency_days,frequency,monetary,aov,tenure_days,purchase_span_days,orders_per_month,revenue_per_month,clv_4m
0,12346.0,2,11,372.86,33.896364,198,196,1.447368,49.060526,NaN
1,12349.0,43,2,1268.52,634.260000,62,18,0.652174,413.647826,1402.62
2,12355.0,40,1,488.21,488.210000,40,0,0.428571,209.232857,NaN
3,12358.0,23,2,1697.93,848.965000,204,181,0.256410,217.683333,NaN
4,12359.0,8,5,2012.03,402.406000,207,198,0.632911,254.687342,551.33


In [48]:
model_df['clv_4m'] = model_df['clv_4m'].fillna(0)

In [49]:
model_df.head()

,Customer ID,recency_days,frequency,monetary,aov,tenure_days,purchase_span_days,orders_per_month,revenue_per_month,clv_4m
0,12346.0,2,11,372.86,33.896364,198,196,1.447368,49.060526,0.00
1,12349.0,43,2,1268.52,634.260000,62,18,0.652174,413.647826,1402.62
2,12355.0,40,1,488.21,488.210000,40,0,0.428571,209.232857,0.00
3,12358.0,23,2,1697.93,848.965000,204,181,0.256410,217.683333,0.00
4,12359.0,8,5,2012.03,402.406000,207,198,0.632911,254.687342,551.33


In [50]:
assert model_df.shape[0] == features.shape[0]
assert (model_df['clv_4m'] >= 0).all()

In [51]:
model_df.describe()

,Customer ID,recency_days,frequency,monetary,aov,tenure_days,purchase_span_days,orders_per_month,revenue_per_month,clv_4m
count,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000
mean,15350.745546,67.899832,3.138487,1439.870535,382.413590,131.865546,63.677983,0.570769,251.348974,973.076407
std,1685.747311,58.484061,5.022510,5665.290641,502.380332,64.493402,71.991826,0.656659,755.827896,4374.460116
min,12346.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.124481,0.000000,0.000000
25%,13898.500000,20.000000,1.000000,260.430000,175.368333,82.000000,0.000000,0.255319,57.535736,0.000000
50%,15352.000000,51.000000,2.000000,549.080000,292.930000,139.000000,31.000000,0.405405,119.234759,310.180000
75%,16808.000000,104.000000,3.000000,1221.335000,431.938750,199.000000,127.000000,0.663717,244.695205,867.865000
max,18287.000000,211.000000,90.000000,169915.970000,10953.500000,211.000000,211.000000,11.203320,21151.365560,151413.770000


In [52]:
model_df.to_csv('model_input.csv', index=False)